In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_info = df['Info'].tolist()

print('总数量：' + str(len(list_info)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for info in list_info:
    work.put_nowait(info)

def crawler():
    global df
    
    while not work.empty():
        info = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(info,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_div = html.xpath('//section[contains(@aria-label, "Components that come in this kit")]/descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]')

                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [_.xpath('./span[1]/text()')[0] for _ in list_div]
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./span[2]/text()')[0] for _ in list_div]
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//section[contains(@aria-label, "Components that come in this kit")]/descendant::tbody[contains(@class, "listing-inner") and ((contains(@class, "altrow-b-0")) or (contains(@class, "altrow-b-1")))]/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_quantity = [_.xpath('./td[1]/text()')[0].strip() for _ in list_tr]
                
                # = = = = = = = = = = = = = = =
                
                list_note = [_.xpath('./td[2]/text()')[0].strip() for _ in list_tr]
                
                # = = = = = = = = = = = = = = =
                
                list_type_code = [json.loads(_.xpath('./td[contains(@id, "listing_data_essential")]/@value')[0])['parttype'] for _ in list_tr]
                
                # = = = = = = = = = = = = = = =
                
                list_part_key = [json.loads(_.xpath('./td[contains(@id, "listing_data_essential")]/@value')[0])['partkey'] for _ in list_tr]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Info': info,
                                        'Kit_序号': [_+1 for _ in range(len(list_div))],
                                        'Kit_Quantity': list_quantity,
                                        'Kit_Note': list_note,
                                        'Kit_Manufacturer': list_manufacturer,
                                        'Kit_Part_Number': list_part_number,
                                        'Kit_Vehcile': '',
                                        'Kit_Alternate_OE_Part_Numbers': '',
                                        'Kit_Pic': '',
                                        'Kit_Info': ['https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key for type_code, part_key in zip(list_type_code, list_part_key)],
                                        'Kit_Src': '',
                                        'Kit_Type_Code': list_type_code,
                                        'Kit_Part_Key': list_part_key,
                                        'Kit_JOIN': [type_code+';'+part_key for type_code, part_key in zip(list_type_code, list_part_key)]})


                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Info': info}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(info +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./kit_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：447

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045  <->  [ok] - 剩余数量：442
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304960  <->  [ok] - 剩余数量：441
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304972  <->  [ok] - 剩余数量：440
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304976  <->  [ok] - 剩余数量：439
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984  <->  [ok] - 剩余数量：438
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000  <->  [ok] - 剩余数量：437
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304984  <->  [ok] - 剩余数量：436
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658980  <->  [ok] - 剩余数量：435
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304980  <->  [ok] - 剩余数量：434
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722076  <->  [ok] - 剩余数量：433
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570025  <->  [ok] - 剩余数量：432
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304992  <->  [ok] - 剩余数量：431
http

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305092  <->  [ok] - 剩余数量：341
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305104  <->  [ok] - 剩余数量：340
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305100  <->  [ok] - 剩余数量：339
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305068  <->  [ok] - 剩余数量：338
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305096  <->  [ok] - 剩余数量：337
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305080  <->  [ok] - 剩余数量：336
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527936  <->  [ok] - 剩余数量：335
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305112  <->  [ok] - 剩余数量：334
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305116  <->  [ok] - 剩余数量：333
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527940  <->  [ok] - 剩余数量：332
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305156  <->  [ok] - 剩余数量：331
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305184  <->  [ok] - 剩余数量：330
https://www.rock

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492497  <->  [ok] - 剩余数量：241
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305396  <->  [ok] - 剩余数量：240
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305340  <->  [ok] - 剩余数量：239
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158145  <->  [ok] - 剩余数量：238
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305400  <->  [ok] - 剩余数量：237
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195072  <->  [ok] - 剩余数量：236
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195076  <->  [ok] - 剩余数量：235
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305404  <->  [ok] - 剩余数量：234
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158141  <->  [ok] - 剩余数量：233
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158153  <->  [ok] - 剩余数量：232
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158149  <->  [ok] - 剩余数量：231
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305416  <->  [ok] - 剩余数量：230
https://w

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15242457  <->  [ok] - 剩余数量：141
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=7075192  <->  [ok] - 剩余数量：140
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305668  <->  [ok] - 剩余数量：139
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305672  <->  [ok] - 剩余数量：138
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305676  <->  [ok] - 剩余数量：137
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305736  <->  [ok] - 剩余数量：136
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305656  <->  [ok] - 剩余数量：135
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305740  <->  [ok] - 剩余数量：134
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305748  <->  [ok] - 剩余数量：133
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305744  <->  [ok] - 剩余数量：132
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305752  <->  [ok] - 剩余数量：131
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305760  <->  [ok] - 剩余数量：130
https://www.roc

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716020  <->  [ok] - 剩余数量：40
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716024  <->  [ok] - 剩余数量：39
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716032  <->  [ok] - 剩余数量：38
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716036  <->  [ok] - 剩余数量：37
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716040  <->  [ok] - 剩余数量：36
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716008  <->  [ok] - 剩余数量：35
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716048  <->  [ok] - 剩余数量：34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716044  <->  [ok] - 剩余数量：33
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716080  <->  [ok] - 剩余数量：32
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716088  <->  [ok] - 剩余数量：31
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716104  <->  [ok] - 剩余数量：30
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716096  <->  [ok] - 剩余数量：29
https://www.rockauto.com/en/